In [10]:
import warnings
warnings.simplefilter('ignore')

In [11]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
from IPython.display import clear_output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
# import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
import random
from numpy.random import randint, shuffle
from ipywidgets import Checkbox
# !wget https://raw.githubusercontent.com/germanesosa/ipywidget-autocomplete/master/autoFill.py
# import autoFill as af
# from ipywidgets import HTML, HBox

In [12]:
# HTML("""
# <style>
# .output_png {
#     display: table-cell;
#     text-align: center;
#     vertical-align: middle;
# }
# </style>
# """)

# CSS = """
# .output {
#     align-items: center;
# }
# """

# HTML('<style>{}</style>'.format(CSS))

# HTML("""<style>
# .text_cell_render {
# font-family: Times New Roman, serif;
# }
# </style>
# """)

## <center> BB84 protocol with phase covariant cloning

### Step 1: Alice performs encoding 

##### Input the bitstring length (accepts only positive integers) to generate a random bitstring alongwith randomly generated bases for Alice

##### Only equatorial bases are used instead of Z basis as in intercept-resend BB84 attack 

X basis             |  Y basis
:-------------------------:|:-------------------------:
<img src="./images/xbasis_0.png" width="150">        |        <img src="./images/ybasis_0.png" width="150">


In [13]:


fig = go.FigureWidget( layout=go.Layout() )

def generate_bits(N):
    bits = randint(0, 2, N)
    return bits

def generate_bases(N):
    bases = randint(0, 2, N)
    bases = ["X" if bases[i] == 0 else "Y" for i in range(N)]
    return bases

def make_df(length = "5"):
    length = length.strip()
    if length == "":
        return None
    
    df = pd.DataFrame({"Alice bits": generate_bits(int(length)), "Alice bases": generate_bases(int(length))})   

    df.to_csv("temp.csv",index=False)
    
    return df.style.hide_index()


interact(make_df, length='5')
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)



interactive(children=(Text(value='5', description='length'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

### Step 2: Eve eavesdrops using cloning

##### Use the slider to change the cloning angle $\Theta_{1}$
##### Click on the "Clone" button to perform cloning. It also acts as a reset button. 

#### <center> Circuit for cloning qubit 0 in X basis with $\Theta_{1}$ = $\frac{\pi}{4}$             

<center> <img  src="./images/circuit_image.png"> 

In [27]:
slider = widgets.FloatSlider(description=r'$\Theta_{1} ( {\small\times} \frac{\pi}{8})$',min=1.1,max=2.91,step=0.1,value=2)

theta1_in = 2 * (np.pi/8)
theta2_in = 0.5 * np.arccos(np.cos(2*theta1_in)/np.sin(2*theta1_in))
theta3_in = np.arctan(np.tan(theta1_in) * np.tan(theta2_in))

text1 = widgets.FloatText(disabled=True, description=r'$\Theta_{1} (rad)$', value=round(theta1_in,4))
text2 = widgets.FloatText(disabled=True, description=r'$\Theta_{2} (rad)$', value=round(theta2_in,4))
text3 = widgets.FloatText(disabled=True, description=r'$\Theta_{3} (rad)$',value=round(theta3_in,4))


def compute(*ignore):
    text1.value = str(round(slider.value * (np.pi/8),4))
#     text2.value = str(slider.value)
#     text3.value = str(slider.value)
    
    theta1 = slider.value * (np.pi/8)
    if theta1 <= np.pi/4:
        theta2 = 0.5 * np.arccos(np.cos(2*theta1)/np.sin(2*theta1))
    if theta1 > np.pi/4:
        theta2 = 0.5 * np.arccos(-np.cos(2*theta1)/np.sin(2*theta1))
        
    theta3 = np.arctan(np.tan(theta1) * np.tan(theta2))
    
    text2.value = str(round(theta2,4))
    text3.value = str(round(theta3,4))
    
slider.observe(compute, 'value')

# slider.value = 4
# interact(f, x=widgets.IntSlider(min=-10,max=30,step=1,value=10))

widgets.VBox([slider, text1,text2,text3])

In [31]:
# def on_value_change(change):
#     newval = slider.value
# #     n.value = str(int(r.value * fp.value * ne.value * fl.value * fi.value * fc.value * l.value)) + " civilizations" 

# slider.observe(on_value_change)
# # slider.observe(on_value_change)
# print(newval)

In [15]:


# button = widgets.Button(description="Clone!")
# output = widgets.Output()

# display(button, output)


# def on_button_clicked(b):
    
# #     pred_df = wid.result
# #     pred_df = pred_df1.sort_values(by='Predicted returns', ascending=False)
#     with output:
# #         clear_output(wait = False)
# #         display(pred_df.head(10))
#         df1 = pd.read_csv("temp.csv")
#         df1["Eve"] = ["Clones"]*df1.shape[0]
#         clear_output(wait=True)    
#         display(df1.style.hide_index())
    

# button.on_click(on_button_clicked)

In [16]:
# df1 = pd.read_csv("temp.csv")
# df1["Eve"] = ["Clones"]*df1.shape[0]
# df1.style.hide_index()

In [35]:

# print(slider.value,text1.value, text2.value, text3.value)

theta1s_raw = np.linspace(1.1,2.9,19)
# print(theta1s_raw)
theta1s = np.pi/8 * theta1s_raw

probbob_theory = []
probeve_theory = []
errbob_theory = []
erreve_theory = []

for val in theta1s:
    probbob_theory.append(0.5 * (1 + np.sin(2 * val)))
    probeve_theory.append(0.5 * (1 + np.abs(np.cos(2 * val))))
    errbob_theory.append(1 - (0.5 * (1 + np.sin(2 * val))))
    erreve_theory.append(1 - (0.5 * (1 + np.abs(np.cos(2 * val)))))


def computeprob():
    
    slider.observe(compute, 'value')
#     print(slider.value)
    
    idx = list(theta1s_raw).index(slider.value)
    probbob = probbob_theory[idx]
    probeve = probeve_theory[idx]
    
    return probbob, probeve

# print(idx,probbob,probeve)

####    <center> Theoretical Fidelity as a function of $\Theta_{1}$ for Bob and Eve 
    
<center> <img  src="./images/fidelity.png"> 

In [51]:

# plt.figure(figsize=(8,4))
# plt.rcParams.update({'font.size': 22})

# plt.plot(theta1s,probbob_theory,"-o",label="Bob")
# plt.plot(theta1s,probeve_theory,"-o",label="Eve")

# plt.ylim(0.4,1.1)
# plt.xlabel(r"$\Theta_{1}$ (rad)") #($\times \frac{\pi}{8}$)
# plt.ylabel("Fidelity")

# # plt.legend(title="Message bit = 0, X basis")
# plt.show()


In [18]:
button = widgets.Button(description="Clone!")
output = widgets.Output()

display(button, output)


def on_button_clicked(b):
    
#     pred_df = wid.result
#     pred_df = pred_df1.sort_values(by='Predicted returns', ascending=False)
    with output:
#         clear_output(wait = False)
#         display(pred_df.head(10))
        df1 = pd.read_csv("temp.csv")
        df1["Eve"] = ["Clones"]*df1.shape[0]
        clear_output(wait=True)    
        display(df1.style.hide_index())
    

button.on_click(on_button_clicked)

Button(description='Clone!', style=ButtonStyle())

Output()

In [19]:

# box1 = widgets.Checkbox(True, description='Cloning Attack')
# # display(box1)

# box2 = widgets.Checkbox(False, description='Direct Attack')
# # display(box2)

# widgets.HBox([box1, box2])


### Step 3: Bob does sifting and measurement

##### Click on the button below to randomly generate bases for Bob and also display Bob's measurement results after Eve's cloning

In [20]:
# df2 = df1
# df2["Bob bases"] =  generate_bases(df2.shape[0]) 

# def sifting(alice_bases,bob_bases):
#     agreed_base_indices = [i for i in range(len(alice_bases)) if alice_bases[i] == bob_bases[i]]
#     return agreed_base_indices

# sifted_indices = sifting(df2['Alice bases'],df2['Bob bases']) 
# df1['Bob bits'] = None

# try:
#     df2 = df2.drop('Eve_bits', axis=1)
# except:
#     pass

# for i in sifted_indices:
#     if random.random() < probbob:
#         df2['Bob bits'][i] = df2['Alice bits'][i]
#     else:
#         df2['Bob bits'][i] = int(not(df2['Alice bits'][i]))
        

In [38]:

def sifting(alice_bases,bob_bases):
    agreed_base_indices = [i for i in range(len(alice_bases)) if alice_bases[i] == bob_bases[i]]
    return agreed_base_indices

button = widgets.Button(description="Bob measures!")
output = widgets.Output()

display(button, output)

# display(df1.style.hide_index())

def on_button_clicked(b):
    with output: 
        
        df1 = pd.read_csv("temp.csv")
        df1["Eve"] = ["Clones"]*df1.shape[0]
        
        df2 = df1
        df2["Bob bases"] =  generate_bases(df2.shape[0]) 
    
        sifted_indices = sifting(df2['Alice bases'],df2['Bob bases']) 
        df2['Bob bits'] = ""
        
        probbob, probeve = computeprob()
        
        for i in sifted_indices:
            if random.random() < probbob:
                df2['Bob bits'][i] = df2['Alice bits'][i]
            else:
                df2['Bob bits'][i] = int(not(df2['Alice bits'][i]))
        
#         df2.to_csv("temp2.csv",index=False)
        
        clear_output(wait=True)
        display(df2.style.hide_index())

        df3 = df2
        df3['Eve bits'] = ""
        
        for i in sifted_indices:
            if random.random() < probeve:
                df3['Eve bits'][i] = df3['Alice bits'][i]
            else:
                df3['Eve bits'][i] = int(not(df3['Alice bits'][i]))

        df3.to_csv("temp2.csv",index=False)
        
#         print(probbob, " ", probeve)
        
button.on_click(on_button_clicked)       

Button(description='Bob measures!', style=ButtonStyle())

Output()

### Step 4: Eve performs delayed measurement

##### Click on the button below to to display Eve's measurement results as well as QBER for both Bob and Eve. QBER is calculated using all the agreed base indices.

In [22]:
# df2 = pd.read_csv("temp2.csv")
# df3 = df2

# df3['Bob bits'] = df3['Bob bits'].round().astype('Int64')
# df3['Eve bits'] = ""

# sifted_indices = sifting(df3['Alice bases'],df3['Bob bases']) 

# for i in sifted_indices:
#     if random.random() < probeve:
#         df3['Eve bits'][i] = df3['Alice bits'][i]
#     else:
#         df3['Eve bits'][i] = int(not(df3['Alice bits'][i]))

def qber(alice_bits,bob_bits,agreed_base_indices):
#     test_len = int(len(agreed_base_indices)*frac) + 1
#     test_base_indices = sorted(random.sample(agreed_base_indices,test_len))
    
    alice_test_bits = []
    bob_test_bits = []
    
    for i in agreed_base_indices:
        alice_test_bits.append(alice_bits[i])
        bob_test_bits.append(bob_bits[i])
    
    num_error = len([j for j in range(len(alice_test_bits)) if alice_test_bits[j] != bob_test_bits[j]])
    
    try:
        error_rate = num_error / len(alice_test_bits)
    except: 
        return -999
    
    return error_rate
        
button = widgets.Button(description="Measure!")
output = widgets.Output()

display(button, output)

# display(df1.style.hide_index())

def on_button_clicked(b):
    with output: 
        clear_output(wait=True)
        
        df4 = pd.read_csv("temp2.csv")
        df4['Bob bits'] = df4['Bob bits'].round().astype('Int64')
        df4['Eve bits'] = df4['Eve bits'].round().astype('Int64')
        
        indices = df4[df4['Eve bits'].notnull()].index.tolist()
#         print((indices))
        
        display(df4.style.hide_index())

        print("The Quantum Bit Error Rate for Bob is ", np.round(qber(df4['Alice bits'],df4['Bob bits'],indices),4))
        print("The Quantum Bit Error Rate for Eve is ", np.round(qber(df4['Alice bits'],df4['Eve bits'],indices),4))
        
        
        
button.on_click(on_button_clicked)  

Button(description='Measure!', style=ButtonStyle())

Output()

In [151]:
# # Make the app
# ##################################################

# title_html = """
# <h2>Machine Learning Model Evaluation: Visual & Interactive</h2>

# <ul style="line-height: 1.5">
#   <li>A classifier was trained to identify positive targets, here visualized as blue marbles (upper left).
# The ratio of negative (white) and positive (blue) targets can be controlled via the slider.</li>

#   <li>The strength-adjustable classifier then predicts a score from one to zero for each item (upper center and right).</li>
  
#   <li>Based on this score and an adjustable cutoff threshold, each item is classified as either positive or negative.
# Correct or incorrect classification is highlighted with a green or red outline for each circle (lower left). 
# Subsequently, 
# <a href="https://en.wikipedia.org/wiki/Receiver_operating_characteristic"> the ROC curve and the AUC value<a>
# (lower center) as well as
# <a href="https://en.wikipedia.org/wiki/Precision_and_recall"> precision, recall</a> 
# and 
# <a href="https://en.wikipedia.org/wiki/Accuracy_and_precision#In_binary_classification"> accuracy </a> 
# (lower right) can be calculated.</li>
# </ul>
# """

# description_html = """
# <style>
# p {
#     margin-bottom: 1.2em;
#     line-height: 1.5;
# }
# </style>

# <p>The effects of different configurations regarding class imbalance, model strength or cutoff threshold on evaluation metrics can be studied.
# Also, the relationship between these metrics, e.g. the tradeoff between precision and recall, can be observed.</p>

# <p>This allows e.g. to explore the problem with accuracy for unbalanced classes: In the case of few positive targets, 
# a weak classifier with a high threshold will yield a high accuracy simply due to labelling everything negative.</p>

# <p>Source code <a href="https://github.com/dhaitz/machine-learning-interactive-visualization"> here</a>. Ideas, suggestions and improvements welcome! /<a href="https://dhaitz.github.io">dh</a></p>
# """
# app_contents = [widgets.HTML(title_html, layout=widgets.Layout(margin='0 0 3em 0', max_width='800px')),
#             widgets.HTML(description_html, layout=widgets.Layout(margin='3em 0 0 0', max_width='800px'))]
# app = widgets.VBox(app_contents, layout=widgets.Layout(max_width='1024px', margin='0 auto 0 auto'))

In [150]:
# display(app)

#### <center> [Github](https://github.com/hardikroutray/BB84)